# Adversarial Attack and Defense Mechanisms on Traffic Signs

In [1]:
# imports
from dataset_train import GTSRB_Train
from dataset_test import GTSRB_Test
import torch
from torch.utils.data import DataLoader
import torchvision.transforms as transforms
import torch.nn.functional as F
from evaluation import evaluate
from torch.autograd import Variable
import numpy as np
import matplotlib.pyplot as plt

c:\Users\anarg\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Loading the Dataset

Load data set and perform preprocessing tasks

In [2]:
training_root = f'GTSRB/Training'
testing_root = f'GTSRB/Final_Test/Images'
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
seed = 1
torch.manual_seed(seed)

train_dataset = GTSRB_Train(root=training_root, transform=transforms.Compose([
    transforms.Resize([48, 48]),
    transforms.ToTensor()
]))

test_dataset = GTSRB_Test(root=testing_root, transform=transforms.Compose([
    transforms.Resize([48, 48]),
    transforms.ToTensor()
]))

print(f"Train Size: {len(train_dataset)}")
print(f"Test Size: {len(test_dataset)}")

train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

cuda
Train Size: 26640
Test Size: 12630


## MicronNet

Loading model and setting hyper-parameters

In [ ]:
from MicronNet import MicronNet

batch_size = 64
lr = 0.007
momentum = 0.8
decay = 0.9
l2_norm = 0.00001

model = MicronNet().to(device)
optimizer = torch.optim.SGD(model.parameters(
), lr=lr, momentum=momentum, weight_decay=l2_norm, nesterov=True)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=decay)

epochs = [1]
microNet_evaluations = []
for e in epochs:
    microNet_evaluations.append(evaluate(e, train_loader, test_loader, model,
             optimizer, device, True, scheduler))

## AlexNet

Loading model and setting hyper-parameters

In [ ]:
from AlexNet import AlexNet

model = AlexNet().to(device)
optimizer = torch.optim.Adam(model.parameters())
loss_fn = F.nll_loss

epochs = [5, 10, 15]
alexNet_evaluations = []
for e in epochs:
    alexNet_evaluations.append(alexNet_evaluations.append(evaluate(e, train_loader, test_loader, model,
                                                                   optimizer, device, False)))


## Attack 1: K-Pixel

In [ ]:
from KPixel import k_pixel_attack_sample
attacks = k_pixel_attack_sample(test_loader,model,5)

## Attack 2: K-Pixel Contiguous

In [ ]:
from KPixelContiguous import k_pixel_contiguous_sample, k_pixel_contiguous_attack1
attacks = k_pixel_attack_sample(test_loader,model,5)

## Attack 3: All Pixel Gaussian 

In [ ]:
from AllPixelGaussian import  all_pixel_gaussian_sample
attacks = all_pixel_gaussian_sample(test_loader,model,5)